In [28]:
import pandas as pd
import numpy as np

# 1. Escreva uma query que retorna a quantidade de linhas na tabela Sales.SalesOrderDetail pelo campo SalesOrderID, desde que tenham pelo menos três linhas de detalhes.
## Acha a quantidade de linhas de detalhes (SalesOrderDetailID) por SalesOrderID.
- Após fazer uma análise exploratória dos dados, foi verificado que **para cada SalesOrderID existe uma SalesOrderDetailID.**
 - Foi feito um agrupamento da coluna SalesOrderID com a função de agregação size() para achar o número de linhas para cada SalesOrderID, **que é o mesmo número de linhas da coluna SalesOrderDetailID.**

In [29]:
def question_1(base1="Sales.SalesOrderDetail.csv"):
    
    # Importa base
    SalesOrderDetail = pd.read_csv(f"../data/{base1}", sep=";")
    
    # Agrupa SalesOrderDetail por SalesOrderID com a função de agregação size()
    grouped_SalesOrderDetail = SalesOrderDetail.groupby('SalesOrderID').size().reset_index(name='Entries')
    
    # Seleciona as linhas com número de registros maior ou igual a 3
    answer1 = grouped_SalesOrderDetail.loc[grouped_SalesOrderDetail["Entries"] >= 3]
    
    # Retorna o tamanho do dataframe answer, que é o resultado esperado
    return len(answer1)

In [30]:
question_1("Sales.SalesOrderDetail.csv")

12757

# 2. Escreva uma query que ligue as tabelas Sales.SalesOrderDetail, Sales.SpecialOfferProduct e Production.Product e retorne os 3 produtos (Name) mais vendidos (pela soma de OrderQty), agrupados pelo número de dias para manufatura (DaysToManufacture).
- Após análise exploratória dos dados, foi observado que a chave comum entre as 3 bases é a coluna **ProductID.**
- As colunas relevantes para alcançar o resultado desejado são:
 - **OrderQty**, encontrada somente na tabela Sales.SalesOrderDetail;
 - **DaysToManufacture**, encontrada nas tabelas Sales.SpecialOfferProduct e Production.Product;
 - **Name**, encontrada nas tabelas Sales.SpecialOfferProduct e Production.Product.
- Como DaysToManufacture e Name estão nas duas tabelas Sales.SpecialOfferProduct e Production.Product, e elas possuem os mesmos produtos, inclusive com o mesmo número de registros (504), não é necessário fazer uma relação entre essas duas tabelas.
- Foi feito um merge entre as tabelas Sales.SalesOrderDetail.csv e Production.Product.csv utilizando como chave o ProductID
 - O resultado foi agrupado por DaysToManufacture e Name com a função de agregação sum na coluna OrderQty.
 - Foram selecionados os 3 produtos mais vendidos.
 
 # ALTERAR DESCRIÇÃO

In [298]:
# def question_2(base1="Sales.SalesOrderDetail.csv", base2="Sales.SpecialOfferProduct.csv",
#                base3="Production.Product.csv"):
    
#     # Importa bases
#     SalesOrderDetail = pd.read_csv(f"../data/{base1}", sep=";")
#     SpecialOfferProduct = pd.read_csv(f"../data/{base3}", sep=";")
#     Product = pd.read_csv(f"../data/{base3}", sep=";")
    
#     # Não é necessário fazer o merge entre SpecialOfferProduct e Product.
#     # Não foi executado, mas, como foi solicitado no enunciado, segue:
#     # SpecialOfferProduct_Product = pd.merge(SpecialOfferProduct, Product, on="ProductID", how="inner")
    
#     # Relaziona Sales.SalesOrderDetail e Production.Product para trazer a coluna OrderQty
#     Product_SalesOrderDetail = pd.merge(Product, SalesOrderDetail[["OrderQty", "ProductID"]], on="ProductID", how="inner")
    
#     # Seleciona as colunas relevantes para diminuir o dataframe
#     Product_SalesOrderDetail = Product_SalesOrderDetail[["ProductID", "Name", "OrderQty", "DaysToManufacture"]]
    
#     # Agrupa o dataframe e aplica sum na coluna OrderQty
#     Product_SalesOrderDetail = Product_SalesOrderDetail.groupby(["DaysToManufacture", "Name"]).agg({'OrderQty':sum,})
    
#     # Seleciona os 3 produtos mais vendidos, retorna o nome agrupado por DaysToManufacture
#     answer2 = Product_SalesOrderDetail.groupby('DaysToManufacture').apply(lambda x: x.nlargest(3,"OrderQty")).reset_index(level=0, drop=True)
    

#     return answer2


def question_2(base1="Sales.SalesOrderDetail.csv", base2="Sales.SpecialOfferProduct.csv",
               base3="Production.Product.csv"):
    
    # Importa bases
    SalesOrderDetail = pd.read_csv(f"../data/{base1}", sep=";")
    SpecialOfferProduct = pd.read_csv(f"../data/{base3}", sep=";")
    Product = pd.read_csv(f"../data/{base3}", sep=";")
    
    # Não é necessário fazer o merge entre SpecialOfferProduct e Product.
    # Nesse caso não é necessário
    SpecialOfferProduct_Product = pd.merge(SpecialOfferProduct, Product, on="ProductID", how="right")
    
    # Relaziona Sales.SalesOrderDetail e Production.Product para trazer a coluna OrderQty
    Product_SalesOrderDetail = pd.merge(SpecialOfferProduct_Product, SalesOrderDetail[["OrderQty", "ProductID"]], on="ProductID", how="left")
    
    # Seleciona as colunas relevantes para diminuir o dataframe
    Product_SalesOrderDetail = Product_SalesOrderDetail[["ProductID", "Name_x", "OrderQty", "DaysToManufacture_x"]]
    Product_SalesOrderDetail = Product_SalesOrderDetail.rename(columns={"Name_x":"Name",
                                                                        "DaysToManufacture_x":"DaysToManufacture"})
    
    # Agrupa o dataframe e aplica sum na coluna OrderQty
    Product_SalesOrderDetail = Product_SalesOrderDetail.groupby(["DaysToManufacture", "Name"]).agg({'OrderQty':sum,})
    
    # Seleciona os 3 produtos mais vendidos, retorna o nome agrupado por DaysToManufacture
    answer2 = Product_SalesOrderDetail.groupby('DaysToManufacture').apply(lambda x: x.nlargest(3,"OrderQty")).reset_index(level=0, drop=True)
    
    return answer2

In [299]:
question_2()

OrderQty
DaysToManufacture Name                                    
0                 AWC Logo Cap                      8311.0
                  Water Bottle - 30 oz.             6815.0
                  Sport-100 Helmet, Blue            6743.0
1                 LL Road Frame - Black, 52         1581.0
                  ML Mountain Handlebars            1465.0
                  ML Road Frame-W - Yellow, 44      1435.0
2                 HL Mountain Frame - Silver, 38    1206.0
                  ML Road Frame-W - Yellow, 38       895.0
                  HL Mountain Frame - Black, 38      659.0
4                 Mountain-200 Black, 38            2977.0
                  Mountain-200 Black, 42            2664.0
                  Mountain-200 Silver, 38           2394.0

# 3. Escreva uma query ligando as tabelas Person.Person, Sales.Customer e Sales.SalesOrderHeader de forma a obter uma lista de nomes de clientes e uma contagem de pedidos efetuados.
- Nome dos cliente consta na tabela Person.Person
- 

In [296]:
def question_3(base1="Person.Person.csv", base2="Sales.Customer.csv",
               base3="Sales.SalesOrderHeader.csv"):
    
    # Importa bases
    Person = pd.read_csv(f"../data/{base1}", sep=";")
    Customer = pd.read_csv(f"../data/{base3}", sep=";")
    SalesOrderHeader = pd.read_csv(f"../data/{base3}", sep=";")
    
    # Nesse caso não é necessário
    SalesOrderHeader_Customer = pd.merge(Customer, SalesOrderHeader, on="SalesOrderID", how="outer")
    
    
    Customer_Person = pd.merge(SalesOrderHeader_Customer, Person[["FirstName", "MiddleName", "LastName", "BusinessEntityID"]],
                               left_on="CustomerID_x", right_on="BusinessEntityID", how="inner")
    
    Customer_Person["Name"] = Customer_Person["FirstName"]+" "+Customer_Person["MiddleName"]+" "+Customer_Person["LastName"]
    
    Customer_Person = Customer_Person.groupby("Name")["SalesOrderID"].size().sort_values(ascending=False)
    
    
    return Customer_Person.to_frame(name="Number_of_Orders")

In [297]:
question_3()

,Number_of_Orders
Name,
Morgan C Miller,28
Grace R Lewis,27
Natalie M Martin,27
Grace J Lee,27
Marshall M Shen,27
...,...
Kaitlyn R Howard,1
Kaitlyn O Butler,1
Kaitlyn M Russell,1


### Análise resultado 3

In [209]:
Person, Customer, SalesOrderHeader = question_3()

In [210]:
len(Person)

19972

In [217]:
# pd.set_option('max_columns', None)
len(Customer)

31465

In [216]:
len(SalesOrderHeader)

31465

In [222]:
len(pd.merge(Customer, SalesOrderHeader, on="SalesOrderID", how="outer"))

31465

In [223]:
len(Customer.merge(SalesOrderHeader, on='SalesOrderID', how='outer', indicator=True)
     .query('_merge != "both"')
     .drop('_merge', 1))

0

In [111]:
# set(Person.columns).intersection(set(Customer.columns))
# set(Person.columns).intersection(set(SalesOrderHeader.columns))
# set(Customer.columns).intersection(set(SalesOrderHeader.columns))

### Análise resultado 2

In [224]:
SpecialOfferProduct = pd.read_csv("../data/Sales.SpecialOfferProduct.csv", sep=";")
Product = pd.read_csv("../data/Production.Product.csv", sep=";")

In [193]:
SalesOrderDetail.groupby(["ProductID"]).agg({"OrderQty":sum}).sort_values(by=["OrderQty"])

,OrderQty
ProductID,
897,4
942,7
943,8
911,10
927,15
...,...
708,6532
715,6592
711,6743


In [227]:
len(SpecialOfferProduct.merge(Product, on='ProductID', how='outer', indicator=True)
     .query('_merge != "both"')
     .drop('_merge', 1))

209

In [229]:
len(SpecialOfferProduct.merge(Product, on='ProductID', how='right', indicator=True)
     .query('_merge == "right_only"')
     .drop('_merge', 1))

209

In [233]:
len(pd.merge(SpecialOfferProduct, Product, on="ProductID", how="right"))

747